<a href="https://colab.research.google.com/github/karelyrodri/Machine-Learning-CS5262-50/blob/main/CS5262_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 😀

## Background 
The data used consists of a financial information regarding loans and credit with 15,787 records and 12 features

Numeric Columns:
1.   Loan Amount
2.   Credit Score
3.   Annual Income 
4.   Monthly Debt 
5.   Years of Credit History
6.   Credit Card Balance
7.   Credit Card Limit 
8.   Interest Rate

Categorical Columns:
1.   Loan Term Length - [1 year, 3 year, 5 year, 7 year, 10 year, 15 year, 30 year]
2.   Years in Current Job - [1 Year or Less, 2 - 4 years, 5 - 7 years,  8 - 9 years, 10+ years]
3.   Home Ownership - [Rent, Home Mortgage, Home Owner]
4.   Loan Purpose - [Home Improvements, Business Loan, Car Loan, Home Loan,
 Medical Bills, Educational Expenses]








## Project Description

This project will focus on assessing loan/credit risk. This is the probability an individal will be unable to repay their debt obligations. The goal is to predict the percentage of credit risk given the information supplied in each record.

## Performance Metric
*   Precision =  True Positives/(True Positives + False Positives)
*   Recall =  True Positives/(True Positives + False Negatives)
*   F1 Score = 2 x ((Precision x Recall)/(Precision + Recall))

 

## Load Data
**Python imports**


In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

**Read google spreadsheet as a csv**

In [ ]:
data = pd.read_csv("https://docs.google.com/spreadsheets/d/1AvPiqVtKBoKFiX4h3uGFbWBty7ZhNt4FmWU6jrXhb8U/export?format=csv&grid=")
display(data)
data.info()

,Loan_Amount,Term,Interest_Rate,Credit_Score,Years_in_Current_Job,Home_Ownership,Annual_Income,Purpose,Monthly_Debt,Years_of_Credit_History,Credit_Cards_Balance,Credit_Cards_Limit
0,6966,1 year,8.010,673,1 Year or Less,Home Mortgage,54736,Home Improvements,839.28,16.4,2942,3881
1,5878,1 year,5.570,743,1 Year or Less,Home Mortgage,107767,Home Improvements,1420.73,24.6,35963,89459
2,1010,1 year,6.435,745,1 Year or Less,Home Mortgage,75713,Home Improvements,1242.95,18.0,769,17889
3,11291,30 year,7.615,717,5 - 7 years,Rent,26198,Business Loan,827.41,16.5,6978,13956
4,24216,3 year,7.135,722,2 - 4 years,Rent,178593,Home Improvements,1029.89,15.3,1405,41309
...,...,...,...,...,...,...,...,...,...,...,...,...
15782,188256,30 year,4.295,741,2 - 4 years,Home Mortgage,136904,Home Loan,1996.51,16.4,17931,23532
15783,4542,1 year,7.825,722,1 Year or Less,Rent,33125,Home Improvements,665.27,11.3,11325,20972
15784,15730,7 year,4.295,725,10+ years,Home Mortgage,60952,Home Improvements,1056.51,18.1,9589,13660
15785,14727,3 year,5.370,745,5 - 7 years,Home Mortgage,78544,Educational Expenses,988.35,13.0,17083,31991


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15787 entries, 0 to 15786
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Loan_Amount              15787 non-null  int64  
 1   Term                     15787 non-null  object 
 2   Interest_Rate            15787 non-null  float64
 3   Credit_Score             15787 non-null  int64  
 4   Years_in_Current_Job     15787 non-null  object 
 5   Home_Ownership           15787 non-null  object 
 6   Annual_Income            15787 non-null  int64  
 7   Purpose                  15787 non-null  object 
 8   Monthly_Debt             15787 non-null  float64
 9   Years_of_Credit_History  15787 non-null  float64
 10  Credit_Cards_Balance     15787 non-null  int64  
 11  Credit_Cards_Limit       15787 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 1.4+ MB
